<a href="https://colab.research.google.com/github/efrsw/hsebio-hw3/blob/added-comments/hsebio_hw3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Загрузка данных

#### Поиск нужного генома

Чтобы найти нужные файлы на ftp-сервере можно воспользоваться [инструкцией](https://www.metagenomics.wiki/tools/fastq/ncbi-ftp-genome-download)

In [1]:
!wget ftp://ftp.ncbi.nlm.nih.gov/genomes/ASSEMBLY_REPORTS/assembly_summary_genbank.txt

--2022-10-13 09:03:41--  ftp://ftp.ncbi.nlm.nih.gov/genomes/ASSEMBLY_REPORTS/assembly_summary_genbank.txt
           => ‘assembly_summary_genbank.txt’
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.12, 130.14.250.13, 2607:f220:41e:250::13, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.12|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /genomes/ASSEMBLY_REPORTS ... done.
==> SIZE assembly_summary_genbank.txt ... 451953280
==> PASV ... done.    ==> RETR assembly_summary_genbank.txt ... done.
Length: 451953280 (431M) (unauthoritative)

assembly_summary_ge 100%[===================>] 431.02M  16.4MB/s    in 27s     

2022-10-13 09:04:11 (15.7 MB/s) - ‘assembly_summary_genbank.txt’ saved [451953280]



In [2]:
!grep -E 'Thalassolituus oleivorans' assembly_summary_genbank.txt | cut -f 20 > ftp_folder.txt
!head -n 1 ftp_folder.txt

https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000/355/675/GCA_000355675.1_ASM35567v1


In [3]:
import gzip

!wget 'https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/355/675/GCF_000355675.1_ASM35567v1/GCF_000355675.1_ASM35567v1_genomic.fna.gz'
!wget 'https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/355/675/GCF_000355675.1_ASM35567v1/GCF_000355675.1_ASM35567v1_genomic.gbff.gz'

!gunzip GCF_000355675.1_ASM35567v1_genomic.fna.gz
!gunzip GCF_000355675.1_ASM35567v1_genomic.gbff.gz

--2022-10-13 09:04:11--  https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/355/675/GCF_000355675.1_ASM35567v1/GCF_000355675.1_ASM35567v1_genomic.fna.gz
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.11, 130.14.250.7, 2607:f220:41e:250::7, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.11|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1168222 (1.1M) [application/x-gzip]
Saving to: ‘GCF_000355675.1_ASM35567v1_genomic.fna.gz’

GCF_000355675.1_ASM 100%[===================>]   1.11M  1.06MB/s    in 1.1s    

2022-10-13 09:04:13 (1.06 MB/s) - ‘GCF_000355675.1_ASM35567v1_genomic.fna.gz’ saved [1168222/1168222]

--2022-10-13 09:04:13--  https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/355/675/GCF_000355675.1_ASM35567v1/GCF_000355675.1_ASM35567v1_genomic.gbff.gz
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.11, 130.14.250.7, 2607:f220:41e:250::7, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.ni

### Reading sequences

In [4]:
!pip install bio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 270 kB 4.1 MB/s 
     |████████████████████████████████| 2.6 MB 43.0 MB/s 


In [5]:
from Bio import SeqIO
import numpy as np
import pandas as pd
from itertools import product
import json

seq = None
gb = None

for record in SeqIO.parse("/content/GCF_000355675.1_ASM35567v1_genomic.fna", "fasta"):
    seq = record.seq

for record in SeqIO.parse('/content/GCF_000355675.1_ASM35567v1_genomic.gbff', 'genbank'):
  gb = record

In [6]:
[print(i) for i in SeqIO.parse("/content/GCF_000355675.1_ASM35567v1_genomic.fna", "fasta")]

ID: NC_020888.1
Name: NC_020888.1
Description: NC_020888.1 Thalassolituus oleivorans MIL-1, complete sequence
Number of features: 0
Seq('TTGGGTAGAATGGCTGGCTCGATTGAAATCTTTGATGACGCTAGTAACAGGGGT...CTC')


[None]

In [7]:
[print(i) for i in SeqIO.parse('/content/GCF_000355675.1_ASM35567v1_genomic.gbff', 'genbank')]

ID: NC_020888.1
Name: NC_020888
Description: Thalassolituus oleivorans MIL-1, complete sequence
Database cross-references: BioProject:PRJNA224116, BioSample:SAMEA2272589, Assembly:GCF_000355675.1
Number of features: 7209
/molecule_type=DNA
/topology=circular
/data_file_division=CON
/date=14-DEC-2021
/accessions=['NC_020888']
/sequence_version=1
/keywords=['RefSeq', 'complete genome']
/source=Thalassolituus oleivorans MIL-1
/organism=Thalassolituus oleivorans MIL-1
/taxonomy=['Bacteria', 'Proteobacteria', 'Gammaproteobacteria', 'Oceanospirillales', 'Oceanospirillaceae', 'Thalassolituus']
/references=[Reference(title='Genome Sequence of Thalassolituus oleivorans MIL-1 (DSM 14913T)', ...), Reference(title='Direct Submission', ...)]
/comment=REFSEQ INFORMATION: The reference sequence is identical to
HF680312.1.
The annotation was added by the NCBI Prokaryotic Genome Annotation
Pipeline (PGAP). Information about PGAP can be found here:
https://www.ncbi.nlm.nih.gov/genome/annotation_prok/
CO

[None]

In [8]:
# [)
# strand is reversed automatically
# only seqs that starts with ATG taken

start, end = gb.features[0].location.start, None

ncod = []
cod = []

for i in range(len(gb.features)):
  feature = gb.features[i]
  fseq = feature.extract(seq)

  end = feature.location.start

  if feature.type == 'gene' and fseq[:3] == 'ATG' and len(fseq) % 3 == 0:
    cod.append(fseq.__str__())

    if len(seq[start:end]) != 0:
      ncod.append(seq[start:end].__str__())

    start = feature.location.end

In [9]:
cod[:5]

['ATGACGCTAGTAACAGGGGTAACCGTGACGGGTTTGTGGCAAGACTGCCTTGGGCATTTACAGCATGAGTTGCCTGCGCAGCAATACAACACATGGATCAGACCTTTGATCGCGAGTACGGAAAACGGTCAGTTAGTTCTGAGCGCCCCTAATCGCTTTGTCAAAGATTGGGTTAAAGACAAGTATCTGCAGCGTATTCAAGAAATCTTATCCGAACTTAACGGCGGTCGTATCACTCATGTCGATGTGACTGCGGGTGAGTCGCGTCCTATGTTTAGTCCTCAGGCTGCGCCAAGACCAGAACCACGGCCTGCGGCGTCGTCGGTTGAAGGATTTGCTTTTGCTGCGCCTCGAGTGGAAGCTGAAGAACCAACCAGTACTTTTTCACCGATTGCATCTTCACCGTTAAAAGAATCACCTTCTACTAATAACAACAATGAGTTTGGACGTCAGTCTTCTTCTAATCTCATTTTGCCTGGTCAGGCTTCTTTTAATACTGATCCTATGCCGTCGGCTCCGGTTTCTAATAAACCGAAACGTAATGTGCAGGTTGAAGGTGGTATTCAGCATCAGAGTTTCTTGAATTCGACATTTACCTTTAAAACCTTTGTTGAAGGTAAATCAAACCAGCTAGCGCATGCCGCAGCTAAGCAGATTGCTGAAAATGCCGGTGGTTCTTACAACCCTCTATTTATATATGGTGGTGTCGGTTTAGGTAAAACCCACTTAATGCATGCGGTAGGTAATCATCTGCAAAGTAAAAATCCGAATGCAAAAGTACTTTATGTGCATTCAGAACGCTTTGTGCAAGACATGGTAAAAGCATTACAGCTAAATGCGATTAACGAATTTAAGCGCTTTTATCGTGGTTTAGATGCTTTATTAATTGACGATATTCAATTTTTTGCGGGCAAAGAACGTTCGCAAGAAGAGTTTTTCCATACCTTTAACGCTTTGTTAGAGGGTGGTCAGCAAATGATTTTAACCTGTGATCGCTA

In [10]:
ncod[:5]

['TTGGGTAGAATGGCTGGCTCGATTGAAATCTTTG',
 'ATCTGAGTGGAGAATC',
 'TATTACTTCCTGAT',
 'GCCCATTTTGTAGGAACACGCGCAAGCGGGCAGATATCTAGCGCAATACGACAAGACGCTAGAGCACCGAATTAATTATGAGGAACACCACGA',
 'TCTGAGTTTGTTGTGGGGAAGGCAGCCAATGGCTGCCTTTTTTGTATTTTGGGTATGAAAAAGCCCGCACTAAGCGGGCCTAATCAAATCATACGTCTTTGGTGGGCTGCCGAAGCAGGGATATCCAAAGCTCACTGAAAAACTGTAAACTTTTACCGCGGATCTGTCAAAGTTAAATAGTGTAAATCAAGGAAGACTT']

### Initial probabilities

In [11]:
def seq_probs(seq):
  return np.array([
      seq.count('T'), seq.count('C'), 
      seq.count('A'), seq.count('G')
    ]) / len(seq)

def cod_probs(seq):
  res = []
  for i in range(3):
    res.append(seq_probs(seq[i::3]))
  return np.array(res)

def make_table1(cod_seqs, ncod_seqs):
  table1 = pd.DataFrame(np.vstack((cod_probs(''.join(cod_seqs)), seq_probs(''.join(ncod_seqs)))).T, 
                       index=[f'P({l})' for l in ['T', 'C', 'A', 'G']], 
                       columns=[f'Codon position {i}' for i in range(1, 4)] + ['Not coding'])
  return table1

In [12]:
make_table1(cod, ncod)

,Codon position 1,Codon position 2,Codon position 3,Not coding
P(T),0.181154,0.296239,0.314999,0.272921
P(C),0.206723,0.227157,0.233407,0.228630
P(A),0.256753,0.301996,0.238216,0.276787
P(G),0.355370,0.174608,0.213379,0.221662


### Transition probabilities

In [13]:
def print_dict(dictionary):
  for key in dictionary.keys():
    print(str(key) + " : " + str(dictionary[key]))

In [19]:
def cod_dprobs(seqs):
  dcounts = dict(
    zip([1, 2, 3], 
        [dict(zip([''.join(pair) for pair in product('TCAG', repeat=2)],
                  [0] * 16)) for i in range(3)])
    )
  
  # # Вывод структуры
  # for key in dcounts.keys():
  #     print(str(key) + " : " + str(dcounts[key]) + "\n")

  for seq in seqs:
    for i in range(1, len(seq)):
      dcounts[i % 3 + 1][seq[i-1:i+1]] += 1
  return get_probs(dcounts)

def ncod_dprobs(seqs):
    dcounts =dict(zip([''.join(pair) for pair in product('TCAG', repeat=2)],
                    [0] * 16))
    
    # # Вывод структуры
    # print("0 : " + str(dcounts) + '\n')
    
    for seq in seqs:
      for i in range(1, len(seq)):
        dcounts[seq[i-1:i+1]] += 1
      
    return get_probs({0: dcounts})[0]

def get_probs(dcounts):
  # print_dict(dcounts)
  
  for pos in dcounts:
    nuc_groups = dict(zip('TCAG', [0] * 4))
    for dup in dcounts[pos]:
      nuc_groups[dup[0]] += dcounts[pos][dup]
    for dup in dcounts[pos]:
      dcounts[pos][dup] /= nuc_groups[dup[0]]

    # print(f"Total single nucleotide count for position {pos}:\n{nuc_groups}")

  return dcounts

def make_table2(cod_seqs, ncod_seqs):
  table2 = pd.DataFrame(cod_dprobs(cod_seqs))
  table2[4] = pd.Series(ncod_dprobs(ncod_seqs))
  table2.rename(columns=dict(zip(np.arange(1, 5), 
                                 [f'Codon position {i}' for i in range(1, 4)] + ['Not coding'])), 
                inplace=True)
  return table2

In [15]:
_ = ncod_dprobs(ncod)

0 : {'TT': 68888, 'TC': 45141, 'TA': 52845, 'TG': 49618, 'CT': 45800, 'CC': 38703, 'CA': 52020, 'CG': 44768, 'AT': 59200, 'AC': 45074, 'AA': 70410, 'AG': 44953, 'GT': 42696, 'GC': 52752, 'GA': 44281, 'GG': 36375}
Total single nucleotide count for position 0:
{'T': 216492, 'C': 181291, 'A': 219637, 'G': 176104}


In [16]:
_ = cod_dprobs(cod)

1 : {'TT': 60956, 'TC': 62278, 'TA': 81781, 'TG': 123389, 'CT': 44983, 'CC': 38019, 'CA': 67212, 'CG': 93122, 'AT': 44260, 'AC': 47705, 'AA': 64543, 'AG': 89228, 'GT': 38665, 'GC': 67520, 'GA': 51037, 'GG': 64757}
2 : {'TT': 94160, 'TC': 37638, 'TA': 32530, 'TG': 24536, 'CT': 53580, 'CC': 42788, 'CA': 67899, 'CG': 51255, 'AT': 89509, 'AC': 57280, 'AA': 87667, 'AG': 33225, 'GT': 71599, 'GC': 99119, 'GA': 126754, 'GG': 73024}
3 : {'TT': 97607, 'TC': 55626, 'TA': 68318, 'TG': 87297, 'CT': 63928, 'CC': 57839, 'CA': 57840, 'CG': 57218, 'AT': 96940, 'AC': 61023, 'AA': 104180, 'AG': 52707, 'GT': 69931, 'GC': 68853, 'GA': 18017, 'GG': 25239}
Total single nucleotide count for position 1:
{'T': 328404, 'C': 243336, 'A': 245736, 'G': 221979}
Total single nucleotide count for position 2:
{'T': 188864, 'C': 215522, 'A': 267681, 'G': 370496}
Total single nucleotide count for position 3:
{'T': 308848, 'C': 236825, 'A': 314850, 'G': 182040}


In [20]:
make_table2(cod, ncod)

,Codon position 1,Codon position 2,Codon position 3,Not coding
TT,0.185613,0.498560,0.316036,0.318201
TC,0.189638,0.199286,0.180108,0.208511
TA,0.249026,0.172240,0.221203,0.244097
TG,0.375723,0.129914,0.282654,0.229191
CT,0.184860,0.248606,0.269938,0.252633
CC,0.156241,0.198532,0.244227,0.213486
CA,0.276211,0.315044,0.244231,0.286942
CG,0.382689,0.237818,0.241605,0.246940
AT,0.180112,0.334387,0.307893,0.269536
AC,0.194131,0.213986,0.193816,0.205220


In [18]:
make_table2(cod, ncod)

1 : {'TT': 60956, 'TC': 62278, 'TA': 81781, 'TG': 123389, 'CT': 44983, 'CC': 38019, 'CA': 67212, 'CG': 93122, 'AT': 44260, 'AC': 47705, 'AA': 64543, 'AG': 89228, 'GT': 38665, 'GC': 67520, 'GA': 51037, 'GG': 64757}
2 : {'TT': 94160, 'TC': 37638, 'TA': 32530, 'TG': 24536, 'CT': 53580, 'CC': 42788, 'CA': 67899, 'CG': 51255, 'AT': 89509, 'AC': 57280, 'AA': 87667, 'AG': 33225, 'GT': 71599, 'GC': 99119, 'GA': 126754, 'GG': 73024}
3 : {'TT': 97607, 'TC': 55626, 'TA': 68318, 'TG': 87297, 'CT': 63928, 'CC': 57839, 'CA': 57840, 'CG': 57218, 'AT': 96940, 'AC': 61023, 'AA': 104180, 'AG': 52707, 'GT': 69931, 'GC': 68853, 'GA': 18017, 'GG': 25239}
Total single nucleotide count for position 1:
{'T': 328404, 'C': 243336, 'A': 245736, 'G': 221979}
Total single nucleotide count for position 2:
{'T': 188864, 'C': 215522, 'A': 267681, 'G': 370496}
Total single nucleotide count for position 3:
{'T': 308848, 'C': 236825, 'A': 314850, 'G': 182040}
0 : {'TT': 68888, 'TC': 45141, 'TA': 52845, 'TG': 49618, 'CT'

,Codon position 1,Codon position 2,Codon position 3,Not coding
TT,0.185613,0.498560,0.316036,0.318201
TC,0.189638,0.199286,0.180108,0.208511
TA,0.249026,0.172240,0.221203,0.244097
TG,0.375723,0.129914,0.282654,0.229191
CT,0.184860,0.248606,0.269938,0.252633
CC,0.156241,0.198532,0.244227,0.213486
CA,0.276211,0.315044,0.244231,0.286942
CG,0.382689,0.237818,0.241605,0.246940
AT,0.180112,0.334387,0.307893,0.269536
AC,0.194131,0.213986,0.193816,0.205220
